<a href="https://colab.research.google.com/github/CiaraFarrellSETU/phd/blob/main/Polardstown_cnn_%26HIREsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.warp import reproject
import geopandas as gpd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from shapely.geometry import mapping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [3]:
rgb_path = "/content/pollardstown_ortho.tif"
ndvi_path = "/content/Pollardstown_Summer_NDVI.tif"
shapefile_path = "/content/Pollardstown_clappied.shp"

In [4]:
rgb_src = rasterio.open(rgb_path)
ndvi_src = rasterio.open(ndvi_path)

In [6]:

scale_factor = 0.1  # Reduce resolution by 90% (adjust as needed)
new_height = int(rgb_src.height * scale_factor)
new_width = int(rgb_src.width * scale_factor)

print(f"Downsampling to {new_width} x {new_height} pixels...")


Downsampling to 4000 x 4000 pixels...


In [7]:

rgb_resampled = np.empty((3, new_height, new_width), dtype=np.float32)
for i in range(1, 4):
    reproject(
        source=rasterio.band(rgb_src, i),
        destination=rgb_resampled[i-1],
        src_transform=rgb_src.transform,
        src_crs=rgb_src.crs,
        dst_transform=rgb_src.transform,
        dst_crs=rgb_src.crs,
        resampling=Resampling.average,
        num_threads=2
    )


In [8]:

ndvi_resampled = np.empty((new_height, new_width), dtype=np.float32)
reproject(
    source=rasterio.band(ndvi_src, 1),
    destination=ndvi_resampled,
    src_transform=ndvi_src.transform,
    src_crs=ndvi_src.crs,
    dst_transform=rgb_src.transform,
    dst_crs=rgb_src.crs,
    resampling=Resampling.average,
    num_threads=2
)


(array([[-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
         -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
        [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
         -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
        [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
         -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
        ...,
        [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
         -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
        [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
         -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
        [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
         -3.4028235e+38, -3.4028235e+38, -3.4028235e+38]], dtype=float32),
 Affine(0.02999680000000226, 0.0, 676477.6457004823,
        0.0, -0.029996799999996437, 717008.3241714993))

In [10]:
stacked = np.vstack([rgb_resampled, ndvi_resampled[np.newaxis, ...]])  # shape: (4, H, W

In [11]:
habitats = gpd.read_file(shapefile_path)

In [13]:

patch_size = 64
X = []
y = []

for idx, row in habitats.iterrows():
    geom = [mapping(row['geometry'])]
    label = row['Id']  # Adjust to your shapefile attribute name

    # Calculate bounding box in downsampled coordinates
    # Convert polygon to raster indices
    bounds = row['geometry'].bounds
    minx, miny, maxx, maxy = bounds

    # Convert to pixel indices
    col_start = int((minx - rgb_src.bounds.left) / (rgb_src.res[0] / (1/scale_factor)))
    col_end = int((maxx - rgb_src.bounds.left) / (rgb_src.res[0] / (1/scale_factor)))
    row_start = int((rgb_src.bounds.top - maxy) / (rgb_src.res[1] / (1/scale_factor)))
    row_end = int((rgb_src.bounds.top - miny) / (rgb_src.res[1] / (1/scale_factor)))

    # Clip from downsampled stack
    clipped = stacked[:, row_start:row_end, col_start:col_end]

    # Create patches
    h, w = clipped.shape[1], clipped.shape[2]
    for i in range(0, h - patch_size, patch_size):
        for j in range(0, w - patch_size, patch_size):
            patch = clipped[:, i:i+patch_size, j:j+patch_size]
            if patch.shape[1] == patch_size and patch.shape[2] == patch_size:
                X.append(patch.transpose(1, 2, 0))  # (H, W, Channels)
                y.append(label)

X = np.array(X)
y = np.array(y)


In [14]:

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

In [17]:

X_train = X_train / 255.0
X_test = X_test / 255.0


In [22]:

num_classes = y_onehot.shape[1]


from tensorflow.keras import Input, layers, models

model = models.Sequential([
    Input(shape=(patch_size, patch_size, 4)),  # Explicit input layer
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [23]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 194ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 205ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 185ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 163ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 184ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 190ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 193ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 164

In [25]:

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")



16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.00


In [28]:
from sklearn.metrics import confusion_matrix

y_pred_onehot = model.predict(X_test)
y_pred = np.argmax(y_pred_onehot, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step
Confusion Matrix:
[[496]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [33]:

import numpy as np

# Check unique classes and their counts
classes, counts = np.unique(y, return_counts=True)

print("Class Distribution:")
for cls, cnt in zip(classes, counts):
    print(f"{cls}: {cnt} samples")

# If you used LabelEncoder, you can map back to class names:
print("\nMapped Class Names:")
for cls, cnt in zip(classes, counts):
    print(f"{encoder.classes_[cls]}: {cnt} samples")


Class Distribution:
15: 2480 samples

Mapped Class Names:


IndexError: index 15 is out of bounds for axis 0 with size 1

In [35]:

import geopandas as gpd

# Load shapefile
shapefile_path = "/content/Pollardstown_clappied.shp"
habitats = gpd.read_file(shapefile_path)

# Check available columns
print("Columns in shapefile:", habitats.columns)

# Verify unique habitat types using 'Id' column
if 'Id' in habitats.columns:
    unique_types = habitats['Id'].unique()
    print("\nUnique Habitat Types:")
    for t in unique_types:
        print("-", t)
    print(f"\nTotal unique habitat types: {len(unique_types)}")
else:
    print("Column 'Id' not found. Please check the correct attribute name.")


Columns in shapefile: Index(['Id', 'NFS_Code', 'NFS_Name', 'County', 'Co_Code', 'Area_sqm',
       'Area_ha', 'Poly_Num', '7140', '7210', '7230', '6410', '6430', '91E0',
       'OtherAnnex', 'PF1', 'PF2', 'PF3', 'FS1', 'FS2', 'GS1', 'GS2', 'GS3',
       'GS4', 'GM1', 'GA1', 'WS1', 'WN2', 'WN4', 'WN6', 'WN7', 'WL1', 'WL2',
       'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'FW4', 'HH3', 'OtherFossi',
       'SumAnnex', 'SumFossit', 'DataQual', 'Anx_Mapped', 'Non_Anx_PF',
       'PRIMARY_FO', 'Anx_perc', 'Anx_perc_l', 'geometry'],
      dtype='object')

Unique Habitat Types:
- 19
- 15
- 10
- 9
- 12
- 2
- 18
- 13
- 20
- 16
- 14
- 17
- 11

Total unique habitat types: 13


HIRE svm

In [1]:



import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.features import rasterize
from shapely.geometry import box
import geopandas as gpd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib


In [2]:

rgb_path = "/content/pollardstown_ortho.tif"
ndvi_path = "/content/Pollardstown_Summer_NDVI.tif"
habitat_shp = "/content/Pollardstown_clappied.shp"


In [5]:

tile_size = 1024  # Adjust based on memory
max_samples_per_tile = 1000
three_class_map = {
    2: 1, 9: 1, 10: 1, 11: 1,           # Fen
    12: 2, 13: 2, 14: 2,                # Grassland
    15: 3, 16: 3, 17: 3, 18: 3, 19: 3, 20: 3 # Woody/Mosaic
}


In [4]:
gdf = gpd.read_file(habitat_shp)

In [7]:

X_samples = []
y_samples = []

with rasterio.open(rgb_path) as src_rgb, rasterio.open(ndvi_path) as src_ndvi:
    for i in range(0, src_rgb.height, tile_size):
        for j in range(0, src_rgb.width, tile_size):
            window = rasterio.windows.Window(j, i, tile_size, tile_size)
            tile_bounds = rasterio.windows.bounds(window, transform=src_rgb.transform)

            # Clip polygons to tile extent
            tile_geom = box(*tile_bounds)
            gdf_tile = gdf[gdf.intersects(tile_geom)]

            if gdf_tile.empty:
                continue  # No training data in this tile

            # Read RGB tile
            rgb_tile = src_rgb.read([1, 2, 3], window=window)
            rgb_tile = np.transpose(rgb_tile, (1, 2, 0))

            # Resample NDVI tile
            ndvi_tile = np.empty((tile_size, tile_size), dtype=np.float32)
            reproject(
                source=rasterio.band(src_ndvi, 1),
                destination=ndvi_tile,
                src_transform=src_ndvi.transform,
                src_crs=src_ndvi.crs,
                dst_transform=src_rgb.window_transform(window),
                dst_crs=src_rgb.crs,
                resampling=Resampling.bilinear
            )
            ndvi_tile = ndvi_tile[..., np.newaxis]

            # Stack RGB + NDVI
            img_tile = np.concatenate([rgb_tile, ndvi_tile], axis=-1)

            # Rasterise polygons for this tile
            shapes = [(geom, value) for geom, value in zip(gdf_tile.geometry, gdf_tile['Id'])]
            mask_tile = rasterize(shapes, out_shape=(tile_size, tile_size),
                                  transform=src_rgb.window_transform(window), fill=0)

            # Apply mapping safely
            mask_tile = np.vectorize(lambda x: three_class_map.get(x, 0))(mask_tile)

            # Extract samples where mask > 0
            rows, cols = np.where(mask_tile > 0)
            if len(rows) > 0:
                # Random sampling to avoid memory overload
                idx = np.random.choice(len(rows), size=min(max_samples_per_tile, len(rows)), replace=False)
                X_samples.append(img_tile[rows[idx], cols[idx]])
                y_samples.append(mask_tile[rows[idx], cols[idx]])

# Combine sampled data
X_samples = np.vstack(X_samples)
y_samples = np.hstack(y_samples)
print("Training samples:", X_samples.shape)


Training samples: (340644, 4)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_samples, y_samples, test_size=0.2, random_state=42)

In [11]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, class_weight='balanced')
clf.fit(X_train, y_train)

ValueError: Floating-point under-/overflow occurred at epoch #1. Scaling input data with StandardScaler or MinMaxScaler might help.

In [ ]:
from sklearn.metrics import classification_repor
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred)


In [20]:

import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.features import rasterize
from shapely.geometry import box
import geopandas as gpd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import joblib


In [21]:

rgb_path = "/content/pollardstown_ortho.tif"
ndvi_path = "/content/Pollardstown_Summer_NDVI.tif"
habitat_shp = "/content/Pollardstown_clappied.shp"

tile_size = 1024  # Adjust based on memory
max_samples_per_tile = 1000  # Limit samples per tile
batch_size = 50000  # For incremental training
three_class_map = {
    2: 1, 9: 1, 10: 1, 11: 1,           # Fen
    12: 2, 13: 2, 14: 2,                # Grassland
    15: 3, 16: 3, 17: 3, 18: 3, 19: 3, 20: 3 # Woody/Mosaic
}


In [15]:
gdf = gpd.read_file(habitat_shp)

In [23]:


X_samples = []
y_samples = []

with rasterio.open(rgb_path) as src_rgb, rasterio.open(ndvi_path) as src_ndvi:
    for i in range(0, src_rgb.height, tile_size):
        for j in range(0, src_rgb.width, tile_size):
            window = rasterio.windows.Window(j, i, tile_size, tile_size)
            tile_bounds = rasterio.windows.bounds(window, transform=src_rgb.transform)

            # Clip polygons to tile extent
            tile_geom = box(*tile_bounds)
            gdf_tile = gdf[gdf.intersects(tile_geom)]

            if gdf_tile.empty:
                continue

            # Read RGB tile
            rgb_tile = src_rgb.read([1, 2, 3], window=window)
            rgb_tile = np.transpose(rgb_tile, (1, 2, 0))

            # Resample NDVI tile
            ndvi_tile = np.empty((tile_size, tile_size), dtype=np.float32)
            reproject(
                source=rasterio.band(src_ndvi, 1),
                destination=ndvi_tile,
                src_transform=src_ndvi.transform,
                src_crs=src_ndvi.crs,
                dst_transform=src_rgb.window_transform(window),
                dst_crs=src_rgb.crs,
                resampling=Resampling.bilinear
            )
            ndvi_tile = ndvi_tile[..., np.newaxis]

            # Stack RGB + NDVI
            img_tile = np.concatenate([rgb_tile, ndvi_tile], axis=-1)

            # Rasterise polygons for this tile
            shapes = [(geom, value) for geom, value in zip(gdf_tile.geometry, gdf_tile['Id'])]
            mask_tile = rasterize(shapes, out_shape=(tile_size, tile_size),
                                  transform=src_rgb.window_transform(window), fill=0)

            # Apply mapping safely
            mask_tile = np.vectorize(lambda x: three_class_map.get(x, 0))(mask_tile)

            # Extract samples where mask > 0
            rows, cols = np.where(mask_tile > 0)
            if len(rows) > 0:
                idx = np.random.choice(len(rows), size=min(max_samples_per_tile, len(rows)), replace=False)
                X_samples.append(img_tile[rows[idx], cols[idx]])
                y_samples.append(mask_tile[rows[idx], cols[idx]])

# Combine sampled data
X_samples = np.vstack(X_samples)
y_samples = np.hstack(y_samples)
print("Collected samples:", X_samples.shape)



Collected samples: (340644, 4)


In [24]:

classes = np.unique(y_samples)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_samples)
class_weight_dict = dict(zip(classes, weights))
print("Class weights:", class_weight_dict)


Class weights: {np.int64(1): np.float64(0.4592882625613811), np.int64(2): np.float64(1.9837523366935133), np.int64(3): np.float64(3.138505763011692)}


In [29]:
scaler = StandardScaler()
clf = SGDClassifier(loss='hinge', max_iter=1000, tol=None, class_weight=class_weight_dict)

# Incremental training in batches
for start in range(0, len(X_samples), batch_size):
    end = start + batch_size
    X_batch = scaler.fit_transform(X_samples[start:end])
    y_batch = y_samples[start:end]
    clf.partial_fit(X_batch, y_batch, classes=classes)

# Save model and scaler
joblib.dump(clf, "sgd_habitat_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [28]:
y_pred_test = clf.predict(scaler.transform(X_test))
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           1       0.73      1.00      0.84     49430
           2       0.00      0.00      0.00     11404
           3       0.00      0.00      0.00      7295

    accuracy                           0.73     68129
   macro avg       0.24      0.33      0.28     68129
weighted avg       0.53      0.73      0.61     68129



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
